In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.circuit.library import HamiltonianGate
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

### import nature library
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

## custom library imports
import os
import sys
import pyscf
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
import time
import inspect
from pprint import pprint
import numpy as np

In [2]:
H = SparsePauliOp.from_list([('II', -1), ('IZ', 0.3), ('XI', -0.3), ('ZY', -0.01), ('YX', 0.1)])
Hmat = H.to_matrix()

# Define the molecule
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Run the driver to get the problem
problem = driver.run()

# Define the log output (here using stdout)
flog = sys.stdout

### Apply freezecore transformer
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)

hamiltonian = problem.hamiltonian.second_q_op()
print("\n".join(str(hamiltonian).splitlines()[:10] + ["..."]))

Fermionic Operator
number spin orbitals=4, number terms=36
  0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.33229086512764805 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.33229086512764805 * ( +_0 +_3 -_3 -_0 )
+ 0.09046559989211567 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211567 * ( +_0 +_1 -_0 -_1 )
+ 0.09046559989211567 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211567 * ( +_0 +_3 -_2 -_1 )
...


In [3]:
eig = np.linalg.eigvals(Hmat)
print('NUMPY eig',eig)

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=H)
print('VQE eig', result.eigenvalue)

NUMPY eig [-0.39328755+1.88514824e-17j -0.89      +1.40385517e-16j
 -1.11      +3.31089849e-17j -1.60671245-2.13162666e-16j]
VQE eig -1.6067123887339632


In [4]:

n_qpe_qbits = 10

U = HamiltonianGate(Hmat, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)
Qpe=qpe.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
measure_circ = measure_circ.compose(Qpe)

measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))
#measure_circ.draw("mpl")


In [5]:
from qiskit_aer import Aer
backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)
result=job.result()
counts = job.result().get_counts()
print(counts)

{'1110000010': 1, '1010000010': 1, '1000100100': 2, '1010110100': 1, '0110000010': 5}


In [6]:
max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')

MAX count: ('0110000010', 5)
Theta value: 0.255859375
QPE-approximated U-eigenvalue: (-0.036807222941358866+0.9993223845883495j)
QPE-approximated H-eigenvalue: -1.607611865704152
